## Count opinion, first-person, analysis

For project analyzing the evaluative content of quotes vs. non-quotes.

This notebook reads in the full CBC csv (in "hardnews") and counts the number of:

* `/first-person`
* `-first-person-`
* `/analysis-`
* `-analysis-`
* `/opinion`

Recall that there were already 10 articles that were parsed as opinion by the GGT. Add that number to the count of opinion from here.

A companion ot this notebook is "extract_opinion.ipynb", which I used month by month for filtering. All of this can probably be merged. At some point.

In [1]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import ast
from ast import literal_eval
import os
import glob

In [2]:
os.chdir(r'C:\Maite\MOD\projects\Monika_Bednarek\Evaluation_quotes\Data\CBC_input\with_fp_analysis_opinion')

## Read original csv

In [3]:
df = pd.read_csv('CBC_hardnews_all_2023.csv', encoding = 'utf8')

In [4]:
df

,_id,url,body,title
0,63b0d066f642cd4562ec7be5,https://www.cbc.ca/sports/hockey/nhl/montreal-...,Alex Ovechkin recorded his 30th career hat tri...,Capitals' offence explodes for 9 goals to send...
1,63b0d065f642cd4562ec7bd0,https://www.cbc.ca/news/world/north-korea-incr...,North Korean leader Kim Jong-un vowed to incre...,Kim Jong-un vows to 'exponentially' increase N...
2,63b0eb8bf642cd4562ed022d,https://www.cbc.ca/news/canada/british-columbi...,Starting Monday Translink will modify 45 bus r...,Translink is changing service on 45 bus routes...
3,63b0f31df642cd4562ed34cc,https://www.cbc.ca/sports/hockey/canada-sweden...,Connor Bedard had four assists to tie Eric Lin...,Canada tops Sweden at men's world juniors to s...
4,63b0faa9f642cd4562ed62b4,https://www.cbc.ca/sports/hockey/nhl/ottawa-se...,"Lucas Raymond had a goal and an assist, and th...",Red Wings score 4 unanswered goals to rally pa...
...,...,...,...,...
52756,6591e7ed58b434ef00c2bf9c,https://www.cbc.ca/news/world/israel-siblings-...,Israeli Maya Regev lay badly wounded in a nond...,Israeli siblings freed from Hamas describe pas...
52757,6591e87458b434ef00c2c26f,https://www.cbc.ca/sports/football/nfl/philade...,James Conner ran for a two-yard touchdown with...,Canadian rookie Sydney Brown returns intercept...
52758,6591f60658b434ef00c3239c,http://www.cbc.ca/news/world/israel-hamas-war-...,Israeli strikes in central Gaza killed at leas...,Israeli strikes in central Gaza kill at least ...
52759,6591f6b158b434ef00c32999,https://www.cbc.ca/news/canada/manitoba/jets-m...,Rutger McGroarty was desperate to play for his...,Jets prospect McGroarty savouring 2nd world ju...


In [54]:
total

53197

In [7]:
n_original = len(df)

In [22]:
n_original

52761

Remember, plus 10 already extracted by GGT. Total number of CBC for 2023 = 52771.

## Extract the right information

Filter out the rows that contain:
* `/first-person`
* `-first-person-`
* `/analysis-`
* `-analysis-`
* `/opinion`



In [9]:
df_no_first = df[~df['url'].str.contains(r'/first-person|-first-person-')]

In [10]:
df_no_first

,_id,url,body,title
0,63b0d066f642cd4562ec7be5,https://www.cbc.ca/sports/hockey/nhl/montreal-...,Alex Ovechkin recorded his 30th career hat tri...,Capitals' offence explodes for 9 goals to send...
1,63b0d065f642cd4562ec7bd0,https://www.cbc.ca/news/world/north-korea-incr...,North Korean leader Kim Jong-un vowed to incre...,Kim Jong-un vows to 'exponentially' increase N...
2,63b0eb8bf642cd4562ed022d,https://www.cbc.ca/news/canada/british-columbi...,Starting Monday Translink will modify 45 bus r...,Translink is changing service on 45 bus routes...
3,63b0f31df642cd4562ed34cc,https://www.cbc.ca/sports/hockey/canada-sweden...,Connor Bedard had four assists to tie Eric Lin...,Canada tops Sweden at men's world juniors to s...
4,63b0faa9f642cd4562ed62b4,https://www.cbc.ca/sports/hockey/nhl/ottawa-se...,"Lucas Raymond had a goal and an assist, and th...",Red Wings score 4 unanswered goals to rally pa...
...,...,...,...,...
52756,6591e7ed58b434ef00c2bf9c,https://www.cbc.ca/news/world/israel-siblings-...,Israeli Maya Regev lay badly wounded in a nond...,Israeli siblings freed from Hamas describe pas...
52757,6591e87458b434ef00c2c26f,https://www.cbc.ca/sports/football/nfl/philade...,James Conner ran for a two-yard touchdown with...,Canadian rookie Sydney Brown returns intercept...
52758,6591f60658b434ef00c3239c,http://www.cbc.ca/news/world/israel-hamas-war-...,Israeli strikes in central Gaza killed at leas...,Israeli strikes in central Gaza kill at least ...
52759,6591f6b158b434ef00c32999,https://www.cbc.ca/news/canada/manitoba/jets-m...,Rutger McGroarty was desperate to play for his...,Jets prospect McGroarty savouring 2nd world ju...


In [37]:
n_no_first = len(df_no_first)

In [38]:
n_no_first

52602

In [39]:
df_no_first_no_analysis = df_no_first[~df_no_first['url'].str.contains(r'/analysis|-analysis-')]

In [40]:
df_no_first_no_analysis

,_id,url,body,title
0,63b0d066f642cd4562ec7be5,https://www.cbc.ca/sports/hockey/nhl/montreal-...,Alex Ovechkin recorded his 30th career hat tri...,Capitals' offence explodes for 9 goals to send...
1,63b0d065f642cd4562ec7bd0,https://www.cbc.ca/news/world/north-korea-incr...,North Korean leader Kim Jong-un vowed to incre...,Kim Jong-un vows to 'exponentially' increase N...
2,63b0eb8bf642cd4562ed022d,https://www.cbc.ca/news/canada/british-columbi...,Starting Monday Translink will modify 45 bus r...,Translink is changing service on 45 bus routes...
3,63b0f31df642cd4562ed34cc,https://www.cbc.ca/sports/hockey/canada-sweden...,Connor Bedard had four assists to tie Eric Lin...,Canada tops Sweden at men's world juniors to s...
4,63b0faa9f642cd4562ed62b4,https://www.cbc.ca/sports/hockey/nhl/ottawa-se...,"Lucas Raymond had a goal and an assist, and th...",Red Wings score 4 unanswered goals to rally pa...
...,...,...,...,...
52756,6591e7ed58b434ef00c2bf9c,https://www.cbc.ca/news/world/israel-siblings-...,Israeli Maya Regev lay badly wounded in a nond...,Israeli siblings freed from Hamas describe pas...
52757,6591e87458b434ef00c2c26f,https://www.cbc.ca/sports/football/nfl/philade...,James Conner ran for a two-yard touchdown with...,Canadian rookie Sydney Brown returns intercept...
52758,6591f60658b434ef00c3239c,http://www.cbc.ca/news/world/israel-hamas-war-...,Israeli strikes in central Gaza killed at leas...,Israeli strikes in central Gaza kill at least ...
52759,6591f6b158b434ef00c32999,https://www.cbc.ca/news/canada/manitoba/jets-m...,Rutger McGroarty was desperate to play for his...,Jets prospect McGroarty savouring 2nd world ju...


In [41]:
n_no_first_no_analysis = len(df_no_first_no_analysis)

In [42]:
n_no_first_no_analysis

52438

In [18]:
df_no_first_no_analysis_no_opinion = df_no_first_no_analysis[~df_no_first_no_analysis['url'].str.contains(r'/opinion')]

In [19]:
df_no_first_no_analysis_no_opinion

,_id,url,body,title
0,63b0d066f642cd4562ec7be5,https://www.cbc.ca/sports/hockey/nhl/montreal-...,Alex Ovechkin recorded his 30th career hat tri...,Capitals' offence explodes for 9 goals to send...
1,63b0d065f642cd4562ec7bd0,https://www.cbc.ca/news/world/north-korea-incr...,North Korean leader Kim Jong-un vowed to incre...,Kim Jong-un vows to 'exponentially' increase N...
2,63b0eb8bf642cd4562ed022d,https://www.cbc.ca/news/canada/british-columbi...,Starting Monday Translink will modify 45 bus r...,Translink is changing service on 45 bus routes...
3,63b0f31df642cd4562ed34cc,https://www.cbc.ca/sports/hockey/canada-sweden...,Connor Bedard had four assists to tie Eric Lin...,Canada tops Sweden at men's world juniors to s...
4,63b0faa9f642cd4562ed62b4,https://www.cbc.ca/sports/hockey/nhl/ottawa-se...,"Lucas Raymond had a goal and an assist, and th...",Red Wings score 4 unanswered goals to rally pa...
...,...,...,...,...
52756,6591e7ed58b434ef00c2bf9c,https://www.cbc.ca/news/world/israel-siblings-...,Israeli Maya Regev lay badly wounded in a nond...,Israeli siblings freed from Hamas describe pas...
52757,6591e87458b434ef00c2c26f,https://www.cbc.ca/sports/football/nfl/philade...,James Conner ran for a two-yard touchdown with...,Canadian rookie Sydney Brown returns intercept...
52758,6591f60658b434ef00c3239c,http://www.cbc.ca/news/world/israel-hamas-war-...,Israeli strikes in central Gaza killed at leas...,Israeli strikes in central Gaza kill at least ...
52759,6591f6b158b434ef00c32999,https://www.cbc.ca/news/canada/manitoba/jets-m...,Rutger McGroarty was desperate to play for his...,Jets prospect McGroarty savouring 2nd world ju...


In [43]:
n_no_first_no_analysis_no_opinion = len(df_no_first_no_analysis_no_opinion)

In [44]:
n_no_first_no_analysis_no_opinion

52325

In [45]:
how_many_first = n_original - n_no_first

In [46]:
how_many_first

159

In [47]:
how_many_analysis = n_no_first - n_no_first_no_analysis

In [48]:
how_many_analysis

164

In [49]:
how_many_opinion = n_no_first_no_analysis - n_no_first_no_analysis_no_opinion

In [50]:
how_many_opinion

113

Plus 10 already removed = 123 opinion

In [51]:
n_filtered = how_many_first + how_many_analysis + how_many_opinion

In [52]:
n_filtered

436

In [55]:
n_original

52761

In [54]:
total

53197

In [56]:
n_filtered

436

In [57]:
n_original - n_filtered

52325

## Sanity check, read the filtered csv

In [58]:
os.chdir(r'C:\Maite\MOD\projects\Monika_Bednarek\Evaluation_quotes\Data\CBC_input')

In [59]:
df2 = pd.read_csv('CBC_news_all_2023.csv', encoding = 'utf8')

In [60]:
df2

,id,url,body,title
0,63b0d066f642cd4562ec7be5,https://www.cbc.ca/sports/hockey/nhl/montreal-...,Alex Ovechkin recorded his 30th career hat tri...,Capitals' offence explodes for 9 goals to send...
1,63b0d065f642cd4562ec7bd0,https://www.cbc.ca/news/world/north-korea-incr...,North Korean leader Kim Jong-un vowed to incre...,Kim Jong-un vows to 'exponentially' increase N...
2,63b0eb8bf642cd4562ed022d,https://www.cbc.ca/news/canada/british-columbi...,Starting Monday Translink will modify 45 bus r...,Translink is changing service on 45 bus routes...
3,63b0f31df642cd4562ed34cc,https://www.cbc.ca/sports/hockey/canada-sweden...,Connor Bedard had four assists to tie Eric Lin...,Canada tops Sweden at men's world juniors to s...
4,63b0faa9f642cd4562ed62b4,https://www.cbc.ca/sports/hockey/nhl/ottawa-se...,"Lucas Raymond had a goal and an assist, and th...",Red Wings score 4 unanswered goals to rally pa...
...,...,...,...,...
52320,6591e7ed58b434ef00c2bf9c,https://www.cbc.ca/news/world/israel-siblings-...,Israeli Maya Regev lay badly wounded in a nond...,Israeli siblings freed from Hamas describe pas...
52321,6591e87458b434ef00c2c26f,https://www.cbc.ca/sports/football/nfl/philade...,James Conner ran for a two-yard touchdown with...,Canadian rookie Sydney Brown returns intercept...
52322,6591f60658b434ef00c3239c,http://www.cbc.ca/news/world/israel-hamas-war-...,Israeli strikes in central Gaza killed at leas...,Israeli strikes in central Gaza kill at least ...
52323,6591f6b158b434ef00c32999,https://www.cbc.ca/news/canada/manitoba/jets-m...,Rutger McGroarty was desperate to play for his...,Jets prospect McGroarty savouring 2nd world ju...


In [61]:
len(df2)

52325